In [ ]:
#Imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline  
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
import statistics 
import seaborn as sns
from IPython.display import display, Math, Latex, display_latex

import pylab
import pandas as pd
import numpy as np
# make the plots (graphs) a little wider by default
pylab.rcParams['figure.figsize'] = (10., 8.)
sns.set(font_scale=1.5)
sns.set_style("white")

from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

!pip install linearmodels
import pandas as pd
import seaborn as sns
import numpy as np
import plotly
import plotly.express as px
import warnings
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sns.set(font_scale=1.5)
sns.set_style("white")
plt.rcParams['figure.figsize'] = (12, 8)

from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf
from linearmodels.panel import compare

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown 1DJC_90BgKiMNdQ6qePgzWPLC33W9vJ5D -O "stringency_final.csv"
df  = pd.read_csv("/content/stringency_final.csv")
df.head()

Downloading...
From: https://drive.google.com/uc?id=1DJC_90BgKiMNdQ6qePgzWPLC33W9vJ5D
To: /content/stringency_final.csv
100% 45.1k/45.1k [00:00<00:00, 56.0MB/s]


,State_Code,Date,StringencyIndex,covid_cases,hate_crimes,hate_tweets
0,AK,2020-02-29,15.328276,0.000000,0.0,0.000000
1,AK,2020-03-31,47.939032,175.064760,0.0,2.735387
2,AK,2020-04-30,86.697333,310.466411,0.0,0.000000
3,AK,2020-05-31,69.355484,147.710891,0.0,2.735387
4,AK,2020-06-30,55.000667,657.860545,0.0,0.000000


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['month_year'] = pd.to_datetime(df['Date']).dt.to_period('M')

In [ ]:
df.head()

,State_Code,Date,StringencyIndex,covid_cases,hate_crimes,hate_tweets,year,month,month_year
0,AK,2020-02-29,15.328276,0.000000,0.0,0.000000,2020,2,2020-02
1,AK,2020-03-31,47.939032,175.064760,0.0,2.735387,2020,3,2020-03
2,AK,2020-04-30,86.697333,310.466411,0.0,0.000000,2020,4,2020-04
3,AK,2020-05-31,69.355484,147.710891,0.0,2.735387,2020,5,2020-05
4,AK,2020-06-30,55.000667,657.860545,0.0,0.000000,2020,6,2020-06


In [ ]:
def add_month_in_period(df):
    # Sort the dataframe by year and month
    df = df.sort_values(by=['year', 'month'])
    
    # Find the minimum year and month
    min_year = df['year'].min()
    min_month = df['month'].min()
    
    # Create an empty list to store the month in period values
    month_in_period = []
    
    # Iterate through the dataframe and calculate the month in period for each row
    for i, row in df.iterrows():
        # Calculate the difference in years between the current row and the minimum year
        year_difference = row['year'] - min_year
        # Calculate the difference in months between the current row and the minimum month
        month_difference = (year_difference * 12) + (row['month'] - min_month)
        month_in_period.append(month_difference)
    
    # Add the month in period values to the dataframe as a new column
    df['month_in_period'] = month_in_period
    
    return df
    
df = add_month_in_period(df)

In [ ]:
df.head()

,State_Code,Date,StringencyIndex,covid_cases,hate_crimes,hate_tweets,year,month,month_year,month_in_period
0,AK,2020-02-29,15.328276,0.000000,0.000000,0.000000,2020,2,2020-02,1
13,AL,2020-02-29,5.368276,0.000000,0.000000,0.406378,2020,2,2020-02,1
26,AR,2020-02-29,5.368276,0.000000,0.000000,0.329976,2020,2,2020-02,1
39,AZ,2020-02-29,10.918621,0.000000,0.000000,0.404236,2020,2,2020-02,1
52,CA,2020-02-29,10.822759,0.863644,0.127006,0.787440,2020,2,2020-02,1


In [ ]:
df.tail()

,State_Code,Date,StringencyIndex,covid_cases,hate_crimes,hate_tweets,year,month,month_year,month_in_period
597,VT,2021-02-28,66.994286,4767.809904,0.000000,0.000000,2021,2,2021-02,13
610,WA,2021-02-28,62.040000,3581.022802,0.389934,0.649890,2021,2,2021-02,13
623,WI,2021-02-28,56.020000,4325.131522,0.000000,0.342897,2021,2,2021-02,13
636,WV,2021-02-28,51.852500,6081.397948,0.000000,0.560291,2021,2,2021-02,13
649,WY,2021-02-28,63.195000,4262.202745,0.000000,1.717245,2021,2,2021-02,13


# **FIXED EFFECTS REGRESSION**

In [ ]:
df2=df.set_index(['State_Code','month_in_period']) # set the index to the state code and the month of interest
panel = PanelOLS.from_formula('hate_tweets ~ 1  + StringencyIndex  + EntityEffects',df2).fit() # run a fixed effects model
print(compare({'Fixed Effects': panel,}, stars=True)) # print the model formatted as a regression table

            Model Comparison           
                          Fixed Effects
---------------------------------------
Dep. Variable               hate_tweets
Estimator                      PanelOLS
No. Observations                    650
Cov. Est.                    Unadjusted
R-squared                        0.0076
R-Squared (Within)               0.0076
R-Squared (Between)              0.0284
R-Squared (Overall)              0.0153
F-statistic                      4.5980
P-value (F-stat)                 0.0324
=====================     =============
Intercept                     0.6868***
                               (7.5565)
StringencyIndex                0.0033**
                               (2.1443)
======================= ===============
Effects                          Entity
---------------------------------------

T-stats reported in parentheses


In [ ]:
df3=df.set_index(['State_Code','month_in_period']) # set the index to the state code and the month of interest
panel = PanelOLS.from_formula('hate_tweets ~ 1  + covid_cases  + EntityEffects',df3).fit() # run a fixed effects model
print(compare({'Fixed Effects': panel,}, stars=True)) # print the model formatted as a regression table

            Model Comparison            
                           Fixed Effects
----------------------------------------
Dep. Variable                hate_tweets
Estimator                       PanelOLS
No. Observations                     650
Cov. Est.                     Unadjusted
R-squared                         0.0250
R-Squared (Within)                0.0250
R-Squared (Between)              -0.0333
R-Squared (Overall)               0.0034
F-statistic                       15.352
P-value (F-stat)                  0.0001
=====================     ==============
Intercept                      0.7829***
                                (22.665)
covid_cases                 1.383e-05***
                                (3.9182)
======================= ================
Effects                           Entity
----------------------------------------

T-stats reported in parentheses


In [ ]:
df4=df.set_index(['State_Code','month_in_period']) # set the index to the state code and the month of interest
panel = PanelOLS.from_formula('hate_crimes ~ 1  + covid_cases  + EntityEffects',df4).fit() # run a fixed effects model
print(compare({'Fixed Effects': panel,}, stars=True)) # print the model formatted as a regression table

            Model Comparison           
                          Fixed Effects
---------------------------------------
Dep. Variable               hate_crimes
Estimator                      PanelOLS
No. Observations                    650
Cov. Est.                    Unadjusted
R-squared                        0.0007
R-Squared (Within)               0.0007
R-Squared (Between)              0.0051
R-Squared (Overall)              0.0019
F-statistic                      0.4100
P-value (F-stat)                 0.5222
=====================     =============
Intercept                     0.0645***
                               (8.4664)
covid_cases                   -4.98e-07
                              (-0.6403)
======================= ===============
Effects                          Entity
---------------------------------------

T-stats reported in parentheses


In [ ]:
df5=df.set_index(['State_Code','month_in_period']) # set the index to the state code and the month of interest
panel = PanelOLS.from_formula('hate_crimes ~ 1  + StringencyIndex  + EntityEffects',df5).fit() # run a fixed effects model
print(compare({'Fixed Effects': panel,}, stars=True)) # print the model formatted as a regression table

            Model Comparison           
                          Fixed Effects
---------------------------------------
Dep. Variable               hate_crimes
Estimator                      PanelOLS
No. Observations                    650
Cov. Est.                    Unadjusted
R-squared                        0.0006
R-Squared (Within)               0.0006
R-Squared (Between)              0.0064
R-Squared (Overall)              0.0023
F-statistic                      0.3801
P-value (F-stat)                 0.5378
=====================     =============
Intercept                      0.0495**
                               (2.4901)
StringencyIndex                  0.0002
                               (0.6165)
======================= ===============
Effects                          Entity
---------------------------------------

T-stats reported in parentheses
